# Fetch Codebase and Models

In [ ]:
import os
os.chdir('/content')
CODE_DIR = 'higan'
!git clone https://github.com/genforce/higan.git $CODE_DIR
os.chdir(f'./{CODE_DIR}')
!mkdir -p models/pretrain/pytorch
!wget https://www.dropbox.com/s/l8i9qpgv82zvi7g/stylegan2_church256_generator.pth?dl=0?dl=1 -O models/pretrain/pytorch/stylegan2_church256_generator.pth --quiet

# Define Utility Functions

In [ ]:
import io
import IPython.display
import cv2
import PIL.Image
import os
import numpy as np
from tqdm import tqdm
import torch

from models.helper import build_generator
from utils.logger import setup_logger
from utils.editor import get_layerwise_manipulation_strength
from utils.editor import manipulate


def imshow(images, col, viz_size=256):
  """Shows images in one figure."""
  num, height, width, channels = images.shape
  assert num % col == 0
  row = num // col

  fused_image = np.zeros((viz_size * row, viz_size * col, channels), dtype=np.uint8)

  for idx, image in enumerate(images):
    i, j = divmod(idx, col)
    y = i * viz_size
    x = j * viz_size
    if height != viz_size or width != viz_size:
      image = cv2.resize(image, (viz_size, viz_size))
    fused_image[y:y + viz_size, x:x + viz_size] = image

  fused_image = np.asarray(fused_image, dtype=np.uint8)
  data = io.BytesIO()
  PIL.Image.fromarray(fused_image).save(data, 'jpeg')
  im_data = data.getvalue()
  disp = IPython.display.display(IPython.display.Image(im_data))
  return disp

def build_model(model_name, logger=None):
  """Builds the generator by model name."""
  model = build_generator(model_name, logger=logger)
  return model

def sample_codes(model, num, seed=0):
  """Samples latent codes randomly."""
  np.random.seed(seed)
  #codes = generator.easy_sample(num)
  latent_codes = model.easy_sample(num=num, latent_space_type='w')
  latent_codes = model.easy_synthesize(latent_codes=latent_codes,
                                       latent_space_type='w',
                                       generate_style=False,
                                       generate_image=False)['wp']
  return latent_codes


# Build Generator

In [ ]:
outdoor_model_name = "stylegan2_church" 
outdoor_model = build_model(outdoor_model_name)

# Sample Codes

In [ ]:
#@title { display-mode: "form", run: "auto" }
num_samples = 4 #@param {type:"slider", min:1, max:8, step:1}
noise_seed = 438 #@param {type:"slider", min:0, max:1000, step:1}

outdoor_latent_codes = sample_codes(outdoor_model, num_samples, noise_seed)
synthesis_kwargs = {'latent_space_type': 'wp'}

images = outdoor_model.easy_synthesize(outdoor_latent_codes, **synthesis_kwargs)['image']
imshow(images, col=num_samples)

# Edit Scene Attributes

In [ ]:
#@title { display-mode: "form", run: "auto" }
attribute_name = 'vegetation' #@param ['clouds', 'vegetation', 'sunny']
path = f'boundaries/{outdoor_model_name}/{attribute_name}_boundary.npy'
try:
    boundary_file = np.load(path, allow_pickle=True).item()
    boundary = boundary_file['boundary']
    manipulate_layers = boundary_file['meta_data']['manipulate_layers']
except ValueError:
    boundary = np.load(path)
    manipulate_layers = '6-11'


strength = [1.0 for _ in range(outdoor_model.num_layers)]

distance = 0 #@param {type:"slider", min:-8.0, max:8.0, step:0.2}
outdoor_codes = manipulate(latent_codes=outdoor_latent_codes,
                     boundary=boundary,
                     start_distance=0,
                     end_distance=distance,
                     step=2,
                     layerwise_manipulation=True,
                     num_layers=outdoor_model.num_layers,
                     manipulate_layers=manipulate_layers,
                     is_code_layerwise=True,
                     is_boundary_layerwise=False,
                     layerwise_manipulation_strength=strength)
images = outdoor_model.easy_synthesize(outdoor_codes[:, 1], latent_space_type='wp')['image']
imshow(images, col=num_samples)